# Stormpy Parametric Instantiator

In [6]:
with open('storm_files/parametric_die.pm') as f:
    print(f.read())

// Knuth's model of a fair die using only fair coins
dtmc

const double p;
const double q;

module die

	// local state
	s : [0..7] init 0;
	// value of the dice
	d : [0..6] init 0;
	
	[] s=0 -> p : (s'=1) + (1-p) : (s'=2);
	[] s=1 -> q : (s'=3) + (1-q) : (s'=4);
	[] s=2 -> q : (s'=5) + (1-q) : (s'=6);
	[] s=3 -> p : (s'=1) + (1-p) : (s'=7) & (d'=1);
	[] s=4 -> p : (s'=7) & (d'=2) + (1-p) : (s'=7) & (d'=3);
	[] s=5 -> p : (s'=7) & (d'=4) + (1-p) : (s'=7) & (d'=5);
	[] s=6 -> p : (s'=2) + (1-p) : (s'=7) & (d'=6);
	[] s=7 -> 1: (s'=7);
	
endmodule

rewards "coin_flips"
	[] s<7 : 1;
endrewards

label "one" = s=7&d=1;
label "two" = s=7&d=2;
label "three" = s=7&d=3;
label "four" = s=7&d=4;
label "five" = s=7&d=5;
label "six" = s=7&d=6;
label "done" = s=7;



In [7]:
import stormpy
prism_program = stormpy.parse_prism_program('storm_files/parametric_die.pm')
formula_str = "P=? [F s=7 & d=2]"
properties = stormpy.parse_properties(formula_str, prism_program)
model = stormpy.build_parametric_model(prism_program, properties)
print(model)
parameters = model.collect_probability_parameters()
for x in parameters:
     print(x)

-------------------------------------------------------------- 
Model type: 	DTMC (sparse)
States: 	13
Transitions: 	20
Reward Models:  none
State Labels: 	3 labels
   * deadlock -> 0 item(s)
   * init -> 1 item(s)
   * ((s = 7) & (d = 2)) -> 1 item(s)
Choice Labels: 	none
-------------------------------------------------------------- 

p
q


In [10]:
import stormpy.pars
instantiator = stormpy.pars.PDtmcInstantiator(model)
point = dict()
for x in parameters:
    point[x] = 0.4
rational_parameter_assignments = dict([[x, stormpy.RationalRF(val)] for x, val in point.items()])
instantiated_model = instantiator.instantiate(rational_parameter_assignments)
result = stormpy.model_checking(instantiated_model, properties[0])
print(result.at(0))

0.11428571428571428


### Symbolic model

In [12]:
result = stormpy.model_checking(model, properties[0])
initial_state = model.initial_states[0]
func = result.at(initial_state)
print(func)

((p)^2 * (q+(-1)))/(p*q+(-1))
